In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGSMITH_ENDPOINT"]

'https://api.smith.langchain.com'

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    ChatPromptTemplate,
                                    PromptTemplate,
                                    MessagesPlaceholder)
from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434/"
model_name = "llama3.2:1b"

llm = ChatOllama(
    base_url = base_url,
    model = model_name
)

template = ChatPromptTemplate.from_template("{prompt}")
chain = template | llm | StrOutputParser()

prompt = "My name is Farrel Satya. I work as an AI Engineer at Google."
response = chain.invoke({"prompt": prompt})
print(response)

Farrel Satya, it's a pleasure to meet you. As an AI Engineer at Google, that's quite an impressive job title. What specific area of AI are you working on or interested in? Are you part of a particular project or team that you're excited about? I'm curious to learn more about your work and how you contribute to the field of artificial intelligence.


In [3]:
prompt = "Do you know my name?"
response = chain.invoke({"prompt": prompt})
print(response)

I'm not capable of knowing your identity or personal details. I'm an AI designed to provide information and assist with tasks, but I don't have the ability to store or access any personal data about individuals. Each time you interact with me, it's a new conversation and I don't retain any information from previous chats.


## RunnableWithHistory

In [4]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [5]:
def get_sql_history(session_id: str):
  """Return the chat history for a given session, backed by SQLite."""
  return SQLChatMessageHistory(
    session_id=session_id,
    connection_string="sqlite:///chat_history.db",   
    table_name="chat_history",
  )

In [13]:
session_id = "alice-chat-001"
history = get_sql_history(session_id=session_id)
history.get_messages()

[HumanMessage(content='Hi, I am Farrel Satya. I work as an AI Engineer.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Farrel Satya. As a conversational AI engineer, you must be working on some exciting projects to power the growth of artificial intelligence.\n\nWhat specific area of AI engineering are you currently focusing on? Are you working on natural language processing, machine learning, or something else? I'm here to help answer any questions you may have or provide guidance on how to overcome challenges.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Farrel Satya, correct?', additional_kwargs={}, response_metadata={})]

In [14]:
history.clear()

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [16]:
chain = prompt | llm | StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
  chain,
  get_session_history=get_sql_history,   
  input_messages_key="question",         
  history_messages_key="history",       
)

In [17]:
session_cfg = {"configurable": {"session_id": session_id}}

In [ ]:
print(chain_with_history.invoke(
    {"question": "Hi, I am Farrel Satya. I work as an AI Engineer."},   # first turn
    config=session_cfg,
))

Farrel Satya, nice to meet you. As an AI Engineer, I'd love to discuss your work with me. What specific aspects of Artificial Intelligence do you find most fascinating or challenging? Are you working on any exciting projects or exploring new areas in AI research?


In [ ]:
print(chain_with_history.invoke(
    {"question": "What is my name?"},  # second turn
    config=session_cfg,
))

Your name is Farrel Satya. Is there something else I can help you with, or would you like to know more about AI Engineering or discuss a particular topic related to AI?


In [ ]:
print(chain_with_history.invoke(
    {"question": "What is my occupation?"},  # third turn
    config=session_cfg,
))

I have some additional information about you. As an AI Engineer at Meta, your work likely involves designing and developing artificial intelligence systems, such as natural language processing (NLP) models or computer vision algorithms. Is that correct?


In [21]:
history.get_messages()

[HumanMessage(content='Hi, I am Farrel Satya. I work as an AI Engineer.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Farrel Satya, nice to meet you. As an AI Engineer, I'd love to discuss your work with me. What specific aspects of Artificial Intelligence do you find most fascinating or challenging? Are you working on any exciting projects or exploring new areas in AI research?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Farrel Satya. Is there something else I can help you with, or would you like to know more about AI Engineering or discuss a particular topic related to AI?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my occupation?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I have some additional information about you. As an AI Engineer at Meta, your work likely involves designing and d